In [25]:
# ! pip install mysql-connector-python
# conda install -c anaconda sqlalchemy
# !pip install recommenders
# conda install -c anaconda pymysql


import numpy as np
import pandas as pd
import mysql.connector
import sqlalchemy

from sklearn.metrics.pairwise import cosine_similarity
from sqlalchemy import types
from sqlalchemy import create_engine
from pathlib import Path

In [26]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd=""
)

sqlEngine= create_engine('mysql+pymysql://root:@localhost:3306/tutorcatdb')

mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")
for x in mycursor:
  print(x)

('epl_db',)
('information_schema',)
('mysql',)
('performance_schema',)
('phpmyadmin',)
('projectdb',)
('spl_db2',)
('testdb',)
('tutorcatdb',)


In [27]:
notes_Rating_DataFrame = pd.read_sql('SELECT NotesID,StudentID,Rating FROM notes_rating',con=sqlEngine)

notes_Rating_DataFrame

,NotesID,StudentID,Rating
0,25,10001,5
1,25,10002,4
2,25,10003,4
3,26,10001,4
4,26,10003,4
5,27,10002,3
6,27,10003,4
7,27,10004,4
8,28,10004,4
9,29,10003,4


In [28]:
print("Notes Number : " + str(notes_Rating_DataFrame['NotesID'].nunique()) )

Notes Number : 7


playHoursAvgDf : col( gameName	playHours)

In [29]:
piv_dataFrame=notes_Rating_DataFrame.pivot_table(index='StudentID', columns='NotesID',values="Rating").fillna(0)

In [30]:
piv_dataFrame

NotesID,25,26,27,28,29,30,31
StudentID,,,,,,,
10001,5.0,4.0,0.0,0.0,0.0,0.0,0.0
10002,4.0,0.0,3.0,0.0,0.0,0.0,3.0
10003,4.0,4.0,4.0,0.0,4.0,4.0,0.0
10004,0.0,0.0,4.0,4.0,0.0,0.0,0.0


In [31]:
def standardize(row):
    new_row = (row - row.mean()) / (row.max() - row.min())
    return new_row

piv_dataFrame = piv_dataFrame.apply(standardize)

In [32]:
piv_dataFrame

NotesID,25,26,27,28,29,30,31
StudentID,,,,,,,
10001,0.35,0.5,-0.6875,-0.25,-0.25,-0.25,-0.25
10002,0.15,-0.5,0.0625,-0.25,-0.25,-0.25,0.75
10003,0.15,0.5,0.3125,-0.25,0.75,0.75,-0.25
10004,-0.65,-0.5,0.3125,0.75,-0.25,-0.25,-0.25


In [33]:
notes_similarity = cosine_similarity(piv_dataFrame.T)
print(len(notes_similarity))

7


In [34]:
notes_similarity_df = pd.DataFrame(notes_similarity,index=piv_dataFrame.columns,columns=piv_dataFrame.columns)

notes_similarity_df

NotesID,25,26,27,28,29,30,31
NotesID,,,,,,,
25,1.000000,0.650945,-0.615462,-0.977140,0.225494,0.225494,0.225494
26,0.650945,1.000000,-0.457496,-0.577350,0.577350,0.577350,-0.577350
27,-0.615462,-0.457496,1.000000,0.440225,0.440225,0.440225,0.088045
28,-0.977140,-0.577350,0.440225,1.000000,-0.333333,-0.333333,-0.333333
29,0.225494,0.577350,0.440225,-0.333333,1.000000,1.000000,-0.333333
30,0.225494,0.577350,0.440225,-0.333333,1.000000,1.000000,-0.333333
31,0.225494,-0.577350,0.088045,-0.333333,-0.333333,-0.333333,1.000000


In [35]:
# df = game_similarity_df.reset_index(index=game_similarity_df.loc[:,0])
df = notes_similarity_df.reset_index()
df = df.set_index('NotesID')
df = df.reset_index()

testFram = pd.melt(df,id_vars=["NotesID"],var_name="NotesID2",value_name='Val')
testFram

testFram.to_sql('ai_table', con=sqlEngine, if_exists='replace',index=False, dtype={"Val": types.Float()})
testFram

,NotesID,NotesID2,Val
0,25,25,1.000000
1,26,25,0.650945
2,27,25,-0.615462
3,28,25,-0.977140
4,29,25,0.225494
5,30,25,0.225494
6,31,25,0.225494
7,25,26,0.650945
8,26,26,1.000000
9,27,26,-0.457496


In [36]:
# def get_similarity_game(notesID,rating):
#
#     similarity_mark = game_similarity_df[game_name]*user_rating
#     similarity_mark = similarity_mark.sort_values(ascending=False)
#
#     return similarity_mark
#
# print(get_similarity_game("Warframe",5))

